---\n## Summary & Quick Reference\n\n### Core Python Essentials:\n1. **Data Structures**: Lists (mutable, ordered), Tuples (immutable, ordered), Sets (mutable, unordered, unique)\n2. **Comprehensions**: List `[x for x in items]`, Dict `{k: v for k, v in items}`, Generator `(x for x in items)`\n3. **Mutability**: Immutable types are hashable and can be dict keys\n4. **Scoping**: LEGB rule (Local, Enclosing, Global, Built-in)\n5. **Decorators**: Functions that wrap other functions for enhancement\n\n### OOP Pillars:\n1. **Inheritance**: `super()` for parent method calls\n2. **Polymorphism**: Same interface, different implementations\n3. **Encapsulation**: `_protected`, `__private`, `@property`\n4. **Abstraction**: Abstract base classes with `@abstractmethod`\n\n### Error Handling Best Practices:\n- Use specific exception types\n- Custom exceptions inherit from appropriate base classes\n- try/except/else/finally blocks\n- Retry mechanisms with exponential backoff\n- Logging for debugging\n\n### Concurrency Guidelines:\n- **Threading**: I/O-bound tasks, GIL limitations\n- **Multiprocessing**: CPU-bound tasks, true parallelism\n- **Asyncio**: I/O-bound with async/await syntax\n- **Queue**: Producer-consumer patterns\n\n### Interview Tips:\n1. Write clean, readable code with proper naming\n2. Handle edge cases and errors\n3. Explain time/space complexity\n4. Use appropriate data structures\n5. Follow Python conventions (PEP 8)\n6. Test your code mentally or with examples\n\n### Common Patterns:\n- Context managers for resource management\n- Generator functions for memory efficiency\n- List/dict comprehensions for concise code\n- Functional programming with `map()`, `filter()`, `reduce()`\n- Decorator patterns for cross-cutting concerns"

In [ ]:
import threading\nimport multiprocessing\nimport asyncio\nimport aiohttp\nimport time\nfrom concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor\nfrom queue import Queue, Empty\nimport requests\n\n# Threading example - I/O bound tasks\ndef fetch_account_balance_sync(account_id: str, delay: float = 1.0) -> dict:\n    \"\"\"Simulate API call to fetch account balance\"\"\"\n    time.sleep(delay)  # Simulate network delay\n    return {\n        'account_id': account_id,\n        'balance': 1000.0 + hash(account_id) % 5000,\n        'timestamp': time.time()\n    }\n\ndef fetch_multiple_accounts_threaded(account_ids: list[str]) -> list[dict]:\n    \"\"\"Fetch multiple account balances using threading\"\"\"\n    results = []\n    threads = []\n    lock = threading.Lock()\n    \n    def worker(account_id):\n        balance_info = fetch_account_balance_sync(account_id, 0.5)\n        with lock:  # Thread-safe access to shared resource\n            results.append(balance_info)\n    \n    # Create and start threads\n    for account_id in account_ids:\n        thread = threading.Thread(target=worker, args=(account_id,))\n        threads.append(thread)\n        thread.start()\n    \n    # Wait for all threads to complete\n    for thread in threads:\n        thread.join()\n    \n    return results\n\n# ThreadPoolExecutor - cleaner threading\ndef fetch_multiple_accounts_executor(account_ids: list[str]) -> list[dict]:\n    \"\"\"Fetch account balances using ThreadPoolExecutor\"\"\"\n    with ThreadPoolExecutor(max_workers=5) as executor:\n        futures = [executor.submit(fetch_account_balance_sync, account_id, 0.5) \n                  for account_id in account_ids]\n        return [future.result() for future in futures]\n\n# CPU-bound task for multiprocessing\ndef calculate_compound_interest(principal: float, rate: float, years: int, compounds_per_year: int = 12) -> float:\n    \"\"\"CPU-intensive calculation\"\"\"\n    # Simulate heavy computation\n    for _ in range(1000000):  # Busy work\n        pass\n    \n    return principal * (1 + rate / compounds_per_year) ** (compounds_per_year * years)\n\ndef process_loan_calculations_multiprocessing(loan_data: list[tuple]) -> list[float]:\n    \"\"\"Process loan calculations using multiprocessing\"\"\"\n    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:\n        futures = [executor.submit(calculate_compound_interest, *data) for data in loan_data]\n        return [future.result() for future in futures]\n\n# Producer-Consumer pattern with Queue\nclass TransactionProcessor:\n    def __init__(self, num_workers: int = 3):\n        self.transaction_queue = Queue()\n        self.result_queue = Queue()\n        self.num_workers = num_workers\n        self.workers = []\n        self.stop_event = threading.Event()\n    \n    def worker(self, worker_id: int):\n        \"\"\"Worker thread that processes transactions\"\"\"\n        while not self.stop_event.is_set():\n            try:\n                transaction = self.transaction_queue.get(timeout=1)\n                if transaction is None:  # Poison pill\n                    break\n                \n                # Process transaction\n                result = self.process_transaction(transaction, worker_id)\n                self.result_queue.put(result)\n                self.transaction_queue.task_done()\n                \n            except Empty:\n                continue\n    \n    def process_transaction(self, transaction: dict, worker_id: int) -> dict:\n        \"\"\"Process a single transaction\"\"\"\n        time.sleep(0.1)  # Simulate processing time\n        return {\n            'transaction_id': transaction['id'],\n            'amount': transaction['amount'],\n            'processed_by': f'worker_{worker_id}',\n            'processed_at': time.time()\n        }\n    \n    def start_workers(self):\n        \"\"\"Start worker threads\"\"\"\n        for i in range(self.num_workers):\n            worker = threading.Thread(target=self.worker, args=(i,))\n            worker.daemon = True\n            worker.start()\n            self.workers.append(worker)\n    \n    def add_transaction(self, transaction: dict):\n        \"\"\"Add transaction to processing queue\"\"\"\n        self.transaction_queue.put(transaction)\n    \n    def get_results(self) -> list[dict]:\n        \"\"\"Get processed results\"\"\"\n        results = []\n        while not self.result_queue.empty():\n            try:\n                results.append(self.result_queue.get_nowait())\n            except Empty:\n                break\n        return results\n    \n    def shutdown(self):\n        \"\"\"Shutdown the processor\"\"\"\n        # Send poison pills\n        for _ in range(self.num_workers):\n            self.transaction_queue.put(None)\n        \n        # Wait for workers to finish\n        for worker in self.workers:\n            worker.join()\n\n# Asyncio example - async/await\nasync def fetch_account_balance_async(session: aiohttp.ClientSession, account_id: str) -> dict:\n    \"\"\"Simulate async API call\"\"\"\n    await asyncio.sleep(0.5)  # Simulate async I/O\n    return {\n        'account_id': account_id,\n        'balance': 1000.0 + hash(account_id) % 5000,\n        'timestamp': time.time()\n    }\n\nasync def fetch_multiple_accounts_async(account_ids: list[str]) -> list[dict]:\n    \"\"\"Fetch multiple accounts using asyncio\"\"\"\n    async with aiohttp.ClientSession() as session:\n        tasks = [fetch_account_balance_async(session, account_id) for account_id in account_ids]\n        return await asyncio.gather(*tasks)\n\n# Benchmarking different approaches\ndef benchmark_approaches():\n    \"\"\"Compare performance of different concurrency approaches\"\"\"\n    account_ids = [f\"ACC{i:03d}\" for i in range(10)]\n    \n    print(\"=== Performance Comparison ===\")\n    \n    # Sequential approach\n    start_time = time.time()\n    sequential_results = [fetch_account_balance_sync(acc_id, 0.5) for acc_id in account_ids]\n    sequential_time = time.time() - start_time\n    print(f\"Sequential: {sequential_time:.2f}s\")\n    \n    # Threading approach\n    start_time = time.time()\n    threaded_results = fetch_multiple_accounts_threaded(account_ids)\n    threaded_time = time.time() - start_time\n    print(f\"Threading: {threaded_time:.2f}s\")\n    \n    # ThreadPoolExecutor\n    start_time = time.time()\n    executor_results = fetch_multiple_accounts_executor(account_ids)\n    executor_time = time.time() - start_time\n    print(f\"ThreadPoolExecutor: {executor_time:.2f}s\")\n    \n    # Asyncio\n    start_time = time.time()\n    async_results = asyncio.run(fetch_multiple_accounts_async(account_ids))\n    async_time = time.time() - start_time\n    print(f\"Asyncio: {async_time:.2f}s\")\n    \n    print(f\"\\nSpeedup vs Sequential:\")\n    print(f\"Threading: {sequential_time/threaded_time:.1f}x\")\n    print(f\"ThreadPoolExecutor: {sequential_time/executor_time:.1f}x\")\n    print(f\"Asyncio: {sequential_time/async_time:.1f}x\")\n\n# Demonstration\nprint(\"=== Concurrency & Parallelism Demonstration ===\")\n\n# CPU-bound multiprocessing example\nprint(\"\\n=== CPU-bound Multiprocessing ===\")\nloan_data = [(10000, 0.05, 10), (25000, 0.04, 15), (50000, 0.06, 20), (15000, 0.055, 8)]\n\nstart_time = time.time()\nsequential_results = [calculate_compound_interest(*data) for data in loan_data]\nsequential_time = time.time() - start_time\n\nstart_time = time.time()\nparallel_results = process_loan_calculations_multiprocessing(loan_data)\nparallel_time = time.time() - start_time\n\nprint(f\"Sequential processing: {sequential_time:.2f}s\")\nprint(f\"Parallel processing: {parallel_time:.2f}s\")\nprint(f\"Speedup: {sequential_time/parallel_time:.1f}x\")\n\n# Producer-Consumer pattern\nprint(\"\\n=== Producer-Consumer Pattern ===\")\nprocessor = TransactionProcessor(num_workers=3)\nprocessor.start_workers()\n\n# Add sample transactions\ntransactions = [\n    {'id': f'TXN{i:03d}', 'amount': 100.0 + i * 50, 'type': 'transfer'}\n    for i in range(15)\n]\n\nfor transaction in transactions:\n    processor.add_transaction(transaction)\n\n# Wait a bit for processing\ntime.sleep(2)\n\n# Get results\nresults = processor.get_results()\nprint(f\"Processed {len(results)} transactions\")\nfor result in results[:3]:  # Show first 3 results\n    print(f\"  {result['transaction_id']}: ${result['amount']} by {result['processed_by']}\")\n\n# Shutdown processor\nprocessor.shutdown()\n\n# Benchmark different approaches\nbenchmark_approaches()"

---\n## Concurrency & Parallelism <a id=\"concurrency\"></a>\n\n### Key Concepts:\n- **Concurrency**: Multiple tasks make progress by sharing CPU time\n- **Parallelism**: Multiple tasks execute simultaneously on multiple CPUs\n- **GIL**: Global Interpreter Lock limits true parallelism in CPython\n- **Threading**: Good for I/O-bound tasks\n- **Multiprocessing**: Good for CPU-bound tasks\n- **Asyncio**: Good for I/O-bound and high-level structured network code"

In [ ]:
import time\nimport random\nimport logging\nfrom typing import Optional\n\n# Custom banking exceptions\nclass BankingError(Exception):\n    \"\"\"Base exception for banking operations\"\"\"\n    pass\n\nclass InsufficientFundsError(BankingError):\n    \"\"\"Raised when account has insufficient funds\"\"\"\n    def __init__(self, balance: float, requested: float):\n        self.balance = balance\n        self.requested = requested\n        super().__init__(f\"Insufficient funds: ${balance:.2f} available, ${requested:.2f} requested\")\n\nclass InvalidAccountError(BankingError):\n    \"\"\"Raised when account number is invalid\"\"\"\n    pass\n\nclass TransactionLimitError(BankingError):\n    \"\"\"Raised when transaction exceeds daily limit\"\"\"\n    pass\n\n# Banking class with comprehensive error handling\nclass BankAccount:\n    def __init__(self, account_number: str, initial_balance: float = 0.0, daily_limit: float = 5000.0):\n        self.account_number = account_number\n        self.balance = initial_balance\n        self.daily_limit = daily_limit\n        self.daily_transactions = 0.0\n    \n    def withdraw(self, amount: float) -> float:\n        \"\"\"Withdraw money with proper error handling\"\"\"\n        try:\n            # Input validation\n            if amount <= 0:\n                raise ValueError(\"Withdrawal amount must be positive\")\n            \n            if amount > self.balance:\n                raise InsufficientFundsError(self.balance, amount)\n            \n            if self.daily_transactions + amount > self.daily_limit:\n                raise TransactionLimitError(f\"Transaction would exceed daily limit of ${self.daily_limit}\")\n            \n            # Successful withdrawal\n            self.balance -= amount\n            self.daily_transactions += amount\n            return self.balance\n            \n        except (ValueError, InsufficientFundsError, TransactionLimitError):\n            # Re-raise custom exceptions\n            raise\n        except Exception as e:\n            # Log unexpected errors\n            logging.error(f\"Unexpected error in withdraw: {e}\")\n            raise BankingError(f\"Transaction failed: {str(e)}\")\n\n# Try/except/else/finally demonstration\ndef process_banking_transaction(account: BankAccount, amount: float) -> dict:\n    \"\"\"Comprehensive transaction processing with all exception handling blocks\"\"\"\n    transaction_log = []\n    \n    try:\n        transaction_log.append(f\"Starting transaction for ${amount}\")\n        \n        # Simulate network delay\n        if random.random() < 0.1:  # 10% chance of network error\n            raise ConnectionError(\"Network timeout during transaction\")\n        \n        new_balance = account.withdraw(amount)\n        transaction_log.append(f\"Withdrawal successful\")\n        \n    except InsufficientFundsError as e:\n        transaction_log.append(f\"Transaction failed: {e}\")\n        return {'status': 'failed', 'error': str(e), 'log': transaction_log}\n    \n    except TransactionLimitError as e:\n        transaction_log.append(f\"Transaction blocked: {e}\")\n        return {'status': 'blocked', 'error': str(e), 'log': transaction_log}\n    \n    except ConnectionError as e:\n        transaction_log.append(f\"Network error: {e}\")\n        return {'status': 'network_error', 'error': str(e), 'log': transaction_log}\n    \n    except Exception as e:\n        transaction_log.append(f\"Unexpected error: {e}\")\n        return {'status': 'system_error', 'error': str(e), 'log': transaction_log}\n    \n    else:\n        # Executed only if no exception occurred in try block\n        transaction_log.append(f\"Transaction completed successfully\")\n        return {'status': 'success', 'balance': new_balance, 'log': transaction_log}\n    \n    finally:\n        # Always executed, regardless of exceptions\n        transaction_log.append(f\"Transaction processing finished at {time.time()}\")\n        logging.info(f\"Transaction processed for account {account.account_number}\")\n\n# Retry with exponential backoff\ndef retry_with_backoff(func, max_retries: int = 3, base_delay: float = 1.0):\n    \"\"\"Retry function with exponential backoff\"\"\"\n    for attempt in range(max_retries):\n        try:\n            return func()\n        except ConnectionError as e:\n            if attempt == max_retries - 1:\n                raise e\n            \n            delay = base_delay * (2 ** attempt) + random.uniform(0, 1)\n            print(f\"Attempt {attempt + 1} failed: {e}. Retrying in {delay:.2f} seconds...\")\n            time.sleep(delay)\n    \n    raise Exception(f\"Max retries ({max_retries}) exceeded\")\n\ndef simulate_api_call():\n    \"\"\"Simulate unreliable API call\"\"\"\n    if random.random() < 0.7:  # 70% chance of failure\n        raise ConnectionError(\"API temporarily unavailable\")\n    return {\"status\": \"success\", \"data\": \"API response\"}\n\n# Financial data processing with error handling\ndef process_financial_pipeline(data: list[dict]) -> dict:\n    \"\"\"Process financial data with comprehensive error handling\"\"\"\n    results = {\n        'processed': [],\n        'errors': [],\n        'summary': {'total': len(data), 'successful': 0, 'failed': 0}\n    }\n    \n    for i, record in enumerate(data):\n        try:\n            # Validate required fields\n            required_fields = ['account', 'amount', 'type']\n            missing_fields = [field for field in required_fields if field not in record]\n            \n            if missing_fields:\n                raise ValueError(f\"Missing required fields: {missing_fields}\")\n            \n            # Process the record\n            processed_record = {\n                'account': record['account'],\n                'amount': float(record['amount']),\n                'type': record['type'],\n                'processed_at': time.time()\n            }\n            \n            # Validate business rules\n            if processed_record['amount'] <= 0:\n                raise ValueError(\"Amount must be positive\")\n            \n            if processed_record['type'] not in ['deposit', 'withdrawal', 'transfer']:\n                raise ValueError(f\"Invalid transaction type: {processed_record['type']}\")\n            \n            results['processed'].append(processed_record)\n            results['summary']['successful'] += 1\n            \n        except (ValueError, TypeError, KeyError) as e:\n            error_info = {\n                'record_index': i,\n                'record': record,\n                'error_type': type(e).__name__,\n                'error_message': str(e)\n            }\n            results['errors'].append(error_info)\n            results['summary']['failed'] += 1\n            \n        except Exception as e:\n            # Log unexpected errors but continue processing\n            logging.error(f\"Unexpected error processing record {i}: {e}\")\n            error_info = {\n                'record_index': i,\n                'record': record,\n                'error_type': 'UnexpectedError',\n                'error_message': str(e)\n            }\n            results['errors'].append(error_info)\n            results['summary']['failed'] += 1\n    \n    return results\n\n# Demonstration\nprint(\"=== Error Handling Demonstration ===\")\n\n# Setup logging\nlogging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')\n\n# Create test account\naccount = BankAccount(\"ACC001\", 1000.0, 2000.0)\n\n# Test various error scenarios\ntest_scenarios = [\n    (500.0, \"Normal withdrawal\"),\n    (1500.0, \"Insufficient funds\"),\n    (-100.0, \"Invalid amount\"),\n    (2500.0, \"Exceeds daily limit\")\n]\n\nfor amount, description in test_scenarios:\n    print(f\"\\nTesting: {description} (${amount})\")\n    result = process_banking_transaction(account, amount)\n    print(f\"Status: {result['status']}\")\n    if 'error' in result:\n        print(f\"Error: {result['error']}\")\n    print(f\"Log entries: {len(result['log'])}\")\n\n# Test retry mechanism\nprint(\"\\n=== Retry Mechanism Test ===\")\ntry:\n    result = retry_with_backoff(simulate_api_call, max_retries=3, base_delay=0.5)\n    print(f\"API call successful: {result}\")\nexcept Exception as e:\n    print(f\"API call failed after retries: {e}\")\n\n# Test data pipeline error handling\nprint(\"\\n=== Data Pipeline Error Handling ===\")\nsample_data = [\n    {'account': 'ACC001', 'amount': '100.0', 'type': 'deposit'},\n    {'account': 'ACC002', 'amount': '-50.0', 'type': 'withdrawal'},  # Invalid amount\n    {'account': 'ACC003', 'type': 'deposit'},  # Missing amount\n    {'account': 'ACC004', 'amount': '200.0', 'type': 'invalid_type'},  # Invalid type\n    {'account': 'ACC005', 'amount': '300.0', 'type': 'transfer'}\n]\n\npipeline_result = process_financial_pipeline(sample_data)\nprint(f\"Pipeline processed {pipeline_result['summary']['total']} records\")\nprint(f\"Successful: {pipeline_result['summary']['successful']}\")\nprint(f\"Failed: {pipeline_result['summary']['failed']}\")\nprint(f\"Errors: {len(pipeline_result['errors'])}\")\n\nfor error in pipeline_result['errors']:\n    print(f\"  Record {error['record_index']}: {error['error_type']} - {error['error_message']}\")"

---\n## Error Handling <a id=\"error-handling\"></a>\n\n### Question 1: What's the difference between exceptions and syntax errors?\n\n**Answer:** \n- **Syntax errors**: Detected before execution, prevent code from running (parsing errors)\n- **Exceptions**: Occur during execution, can be caught and handled with try/except blocks"

In [ ]:
import json\nimport csv\nimport pickle\nfrom pathlib import Path\nfrom typing import Iterator, Any\n\n# 1. Preferred way - Context manager\ndef write_transaction_log(filename: str, transactions: list[dict]) -> None:\n    \"\"\"Write transactions to file using context manager\"\"\"\n    with open(filename, 'w', encoding='utf-8') as file:\n        for transaction in transactions:\n            file.write(f\"{transaction['date']},{transaction['amount']},{transaction['type']}\\n\")\n    # File automatically closed here, even if exception occurs\n\ndef read_transaction_log(filename: str) -> list[str]:\n    \"\"\"Read transactions using context manager\"\"\"\n    try:\n        with open(filename, 'r', encoding='utf-8') as file:\n            return file.readlines()\n    except FileNotFoundError:\n        print(f\"File {filename} not found\")\n        return []\n\n# 2. Reading large CSV files efficiently (streaming)\ndef process_large_csv(filename: str) -> Iterator[dict]:\n    \"\"\"Process large CSV file without loading entire file into memory\"\"\"\n    with open(filename, 'r', encoding='utf-8') as file:\n        reader = csv.DictReader(file)\n        for row in reader:\n            # Process one row at a time - memory efficient\n            yield {\n                'account': row.get('account_number'),\n                'amount': float(row.get('amount', 0)),\n                'balance': float(row.get('balance', 0))\n            }\n\ndef summarize_large_transactions(filename: str, threshold: float = 1000.0) -> dict:\n    \"\"\"Efficiently summarize large transaction file\"\"\"\n    large_transactions = []\n    total_amount = 0.0\n    count = 0\n    \n    for transaction in process_large_csv(filename):\n        if abs(transaction['amount']) > threshold:\n            large_transactions.append(transaction)\n        total_amount += transaction['amount']\n        count += 1\n    \n    return {\n        'large_transactions': large_transactions,\n        'total_processed': count,\n        'total_amount': total_amount,\n        'average': total_amount / count if count > 0 else 0\n    }\n\n# 3. JSON file handling\ndef save_account_data(filename: str, accounts: list[dict]) -> None:\n    \"\"\"Save account data to JSON file\"\"\"\n    with open(filename, 'w', encoding='utf-8') as file:\n        json.dump(accounts, file, indent=2, ensure_ascii=False)\n\ndef load_account_data(filename: str) -> list[dict]:\n    \"\"\"Load account data from JSON file\"\"\"\n    try:\n        with open(filename, 'r', encoding='utf-8') as file:\n            return json.load(file)\n    except (FileNotFoundError, json.JSONDecodeError) as e:\n        print(f\"Error loading JSON: {e}\")\n        return []\n\n# 4. Using pathlib instead of os.path\ndef modern_file_operations() -> None:\n    \"\"\"Demonstrate pathlib usage\"\"\"\n    # Create directories\n    data_dir = Path('banking_data')\n    data_dir.mkdir(exist_ok=True)\n    \n    # File operations\n    log_file = data_dir / 'transactions.log'\n    config_file = data_dir / 'config.json'\n    \n    # Check file existence\n    if not log_file.exists():\n        log_file.write_text('Initial log entry\\n', encoding='utf-8')\n    \n    # Read file\n    if log_file.exists():\n        content = log_file.read_text(encoding='utf-8')\n        print(f\"Log content: {content.strip()}\")\n    \n    # File information\n    if log_file.exists():\n        stat = log_file.stat()\n        print(f\"File size: {stat.st_size} bytes\")\n        print(f\"File extension: {log_file.suffix}\")\n        print(f\"File name: {log_file.name}\")\n        print(f\"Parent directory: {log_file.parent}\")\n\n# 5. Custom context manager for database-like operations\nclass TransactionFileManager:\n    \"\"\"Custom context manager for transaction file operations\"\"\"\n    \n    def __init__(self, filename: str):\n        self.filename = filename\n        self.file = None\n        self.transactions_written = 0\n    \n    def __enter__(self):\n        print(f\"Opening transaction file: {self.filename}\")\n        self.file = open(self.filename, 'a', encoding='utf-8')\n        return self\n    \n    def __exit__(self, exc_type, exc_val, exc_tb):\n        if self.file:\n            self.file.close()\n            print(f\"Closed file. Wrote {self.transactions_written} transactions\")\n        \n        if exc_type is not None:\n            print(f\"Exception occurred: {exc_type.__name__}: {exc_val}\")\n        return False  # Don't suppress exceptions\n    \n    def write_transaction(self, amount: float, account: str, transaction_type: str) -> None:\n        \"\"\"Write a transaction to the file\"\"\"\n        if self.file:\n            from datetime import datetime\n            timestamp = datetime.now().isoformat()\n            self.file.write(f\"{timestamp},{account},{amount},{transaction_type}\\n\")\n            self.file.flush()  # Ensure data is written immediately\n            self.transactions_written += 1\n\n# Demonstration\nprint(\"=== File Handling Demonstration ===\")\n\n# Create sample data\nsample_transactions = [\n    {'date': '2024-01-01', 'amount': 1000.0, 'type': 'deposit'},\n    {'date': '2024-01-02', 'amount': -250.0, 'type': 'withdrawal'},\n    {'date': '2024-01-03', 'amount': 500.0, 'type': 'deposit'}\n]\n\nsample_accounts = [\n    {'account_number': 'ACC001', 'owner': 'Alice', 'balance': 1500.0},\n    {'account_number': 'ACC002', 'owner': 'Bob', 'balance': 2300.0}\n]\n\n# Write and read transaction log\nwrite_transaction_log('transactions.log', sample_transactions)\ntransactions = read_transaction_log('transactions.log')\nprint(f\"Read {len(transactions)} transactions\")\n\n# JSON operations\nsave_account_data('accounts.json', sample_accounts)\nloaded_accounts = load_account_data('accounts.json')\nprint(f\"Loaded {len(loaded_accounts)} accounts\")\n\n# Modern file operations with pathlib\nmodern_file_operations()\n\n# Custom context manager\nprint(\"\\n=== Custom Context Manager ===\")\nwith TransactionFileManager('daily_transactions.log') as manager:\n    manager.write_transaction(100.0, 'ACC001', 'deposit')\n    manager.write_transaction(-50.0, 'ACC001', 'withdrawal')\n    manager.write_transaction(200.0, 'ACC002', 'deposit')\n\n# Clean up created files\nimport os\nfor filename in ['transactions.log', 'accounts.json', 'daily_transactions.log']:\n    if os.path.exists(filename):\n        os.remove(filename)\n        print(f\"Cleaned up: {filename}\")"

---\n## File Handling <a id=\"file-handling\"></a>\n\n### Question 1: What's the preferred way to open and close files in Python?\n\n**Answer:** Use context managers (`with` statement) to ensure proper file closure and resource cleanup, even if exceptions occur."

In [ ]:
class BankAccount:\n    \"\"\"Comprehensive example of common dunder methods\"\"\"\n    \n    def __init__(self, account_number: str, balance: float, owner: str):\n        \"\"\"1. __init__ - Object initialization\"\"\"\n        self.account_number = account_number\n        self.balance = balance\n        self.owner = owner\n    \n    def __str__(self) -> str:\n        \"\"\"2. __str__ - Human-readable string representation\"\"\"\n        return f\"Account {self.account_number}: ${self.balance:.2f} ({self.owner})\"\n    \n    def __repr__(self) -> str:\n        \"\"\"3. __repr__ - Developer-friendly string representation\"\"\"\n        return f\"BankAccount('{self.account_number}', {self.balance}, '{self.owner}')\"\n    \n    def __eq__(self, other) -> bool:\n        \"\"\"4. __eq__ - Equality comparison\"\"\"\n        if not isinstance(other, BankAccount):\n            return NotImplemented\n        return self.account_number == other.account_number\n    \n    def __lt__(self, other) -> bool:\n        \"\"\"5. __lt__ - Less than comparison (enables sorting)\"\"\"\n        if not isinstance(other, BankAccount):\n            return NotImplemented\n        return self.balance < other.balance\n    \n    def __add__(self, amount: float):\n        \"\"\"6. __add__ - Addition operator overloading\"\"\"\n        if isinstance(amount, (int, float)):\n            return BankAccount(self.account_number, self.balance + amount, self.owner)\n        return NotImplemented\n    \n    def __len__(self) -> int:\n        \"\"\"7. __len__ - Length of account number\"\"\"\n        return len(self.account_number)\n    \n    def __getitem__(self, key: str):\n        \"\"\"8. __getitem__ - Dictionary-like access\"\"\"\n        if key == 'balance':\n            return self.balance\n        elif key == 'owner':\n            return self.owner\n        elif key == 'account':\n            return self.account_number\n        else:\n            raise KeyError(f\"Key '{key}' not found\")\n    \n    def __bool__(self) -> bool:\n        \"\"\"9. __bool__ - Truth value testing\"\"\"\n        return self.balance > 0\n    \n    def __call__(self, amount: float) -> str:\n        \"\"\"10. __call__ - Make object callable (function-like)\"\"\"\n        self.balance += amount\n        return f\"Transaction processed: ${amount:.2f}. New balance: ${self.balance:.2f}\"\n\n# Demonstrate dunder methods\nprint(\"=== Dunder Methods Demonstration ===\")\n\n# Create accounts\nacc1 = BankAccount(\"ACC001\", 1500.0, \"Alice\")\nacc2 = BankAccount(\"ACC002\", 2300.0, \"Bob\")\nacc3 = BankAccount(\"ACC001\", 1800.0, \"Alice\")  # Same account number as acc1\n\n# 1. __str__ and __repr__\nprint(f\"str(acc1): {str(acc1)}\")\nprint(f\"repr(acc1): {repr(acc1)}\")\n\n# 2. __eq__ - Equality\nprint(f\"\\nacc1 == acc2: {acc1 == acc2}\")\nprint(f\"acc1 == acc3: {acc1 == acc3}\")  # Same account number\n\n# 3. __lt__ - Comparison (enables sorting)\naccounts = [acc2, acc1, BankAccount(\"ACC003\", 500.0, \"Charlie\")]\naccounts.sort()  # Uses __lt__\nprint(f\"\\nSorted accounts by balance:\")\nfor acc in accounts:\n    print(f\"  {acc}\")\n\n# 4. __add__ - Addition\nnew_acc = acc1 + 200.0\nprint(f\"\\nOriginal: {acc1}\")\nprint(f\"After adding $200: {new_acc}\")\n\n# 5. __len__ - Length\nprint(f\"\\nAccount number length: {len(acc1)}\")\n\n# 6. __getitem__ - Dictionary-like access\nprint(f\"acc1['balance']: ${acc1['balance']:.2f}\")\nprint(f\"acc1['owner']: {acc1['owner']}\")\n\n# 7. __bool__ - Truth testing\nempty_acc = BankAccount(\"ACC004\", 0.0, \"David\")\nprint(f\"\\nacc1 is truthy: {bool(acc1)}\")\nprint(f\"empty_acc is truthy: {bool(empty_acc)}\")\n\n# 8. __call__ - Callable object\nresult = acc1(100.0)  # Add $100 to account\nprint(f\"\\nCalling acc1(100.0): {result}\")\n\n# Additional useful dunder methods\nclass TransactionHistory:\n    \"\"\"Example of more dunder methods\"\"\"\n    \n    def __init__(self):\n        self.transactions = []\n    \n    def __iadd__(self, transaction: str):  # += operator\n        self.transactions.append(transaction)\n        return self\n    \n    def __contains__(self, item: str) -> bool:  # 'in' operator\n        return item in self.transactions\n    \n    def __iter__(self):  # Make iterable\n        return iter(self.transactions)\n    \n    def __len__(self) -> int:\n        return len(self.transactions)\n\nhistory = TransactionHistory()\nhistory += \"Deposit: $100\"\nhistory += \"Withdrawal: $50\"\nhistory += \"Transfer: $200\"\n\nprint(f\"\\nTransaction history length: {len(history)}\")\nprint(f\"Contains 'Deposit': {'Deposit' in history}\")\nprint(\"All transactions:\")\nfor transaction in history:\n    print(f\"  {transaction}\")"

### Question 3: What are dunder methods? Give 5 common examples\n\n**Answer:** Dunder (double underscore) methods are special methods that define how objects behave with built-in operations. They enable operator overloading and integration with Python's data model."

In [ ]:
# Method Resolution Order (MRO) Example\n\nclass BankingService:\n    def process_transaction(self):\n        return \"BankingService: Processing transaction\"\n    \n    def validate_account(self):\n        return \"BankingService: Validating account\"\n\nclass SecurityService:\n    def process_transaction(self):\n        return \"SecurityService: Processing with security checks\"\n    \n    def encrypt_data(self):\n        return \"SecurityService: Encrypting data\"\n\nclass AuditService:\n    def process_transaction(self):\n        return \"AuditService: Processing with audit logging\"\n    \n    def log_transaction(self):\n        return \"AuditService: Logging transaction\"\n\n# Multiple inheritance - order matters!\nclass SecureAuditedBank(SecurityService, AuditService, BankingService):\n    def full_process(self):\n        # Calls the first method found in MRO\n        return self.process_transaction()\n\nclass AuditedSecureBank(AuditService, SecurityService, BankingService):\n    def full_process(self):\n        return self.process_transaction()\n\n# Examine MRO\nprint(\"=== Method Resolution Order ===\")\nprint(f\"SecureAuditedBank MRO: {SecureAuditedBank.__mro__}\")\nprint(f\"AuditedSecureBank MRO: {AuditedSecureBank.__mro__}\")\n\n# Test method resolution\nbank1 = SecureAuditedBank()\nbank2 = AuditedSecureBank()\n\nprint(f\"\\nSecureAuditedBank result: {bank1.full_process()}\")\nprint(f\"AuditedSecureBank result: {bank2.full_process()}\")\n\n# Using super() in multiple inheritance\nclass CooperativeBanking(BankingService):\n    def process_transaction(self):\n        result = super().process_transaction()\n        return f\"Cooperative: {result}\"\n\nclass CooperativeSecurity(SecurityService):\n    def process_transaction(self):\n        result = super().process_transaction()\n        return f\"Cooperative Security: {result}\"\n\nclass CooperativeBank(CooperativeSecurity, CooperativeBanking):\n    def process_transaction(self):\n        result = super().process_transaction()\n        return f\"Final: {result}\"\n\nprint(f\"\\nCooperative MRO: {CooperativeBank.__mro__}\")\ncoop_bank = CooperativeBank()\nprint(f\"Cooperative result: {coop_bank.process_transaction()}\")\n\n# Diamond problem resolution\nclass A:\n    def method(self):\n        return \"A\"\n\nclass B(A):\n    def method(self):\n        return \"B -> \" + super().method()\n\nclass C(A):\n    def method(self):\n        return \"C -> \" + super().method()\n\nclass D(B, C):  # Diamond inheritance\n    def method(self):\n        return \"D -> \" + super().method()\n\nprint(f\"\\nDiamond MRO: {D.__mro__}\")\nd = D()\nprint(f\"Diamond result: {d.method()}\")"

### Question 2: Explain Method Resolution Order (MRO) in Python\n\n**Answer:** MRO is the sequence Python follows to search for methods/attributes in class hierarchy. Uses C3 linearization algorithm to ensure consistent, predictable method resolution in multiple inheritance scenarios."

In [ ]:
from abc import ABC, abstractmethod\n\n# 1. Abstraction - Abstract Base Class\nclass Account(ABC):\n    def __init__(self, account_number: str, balance: float):\n        self._account_number = account_number  # Protected\n        self.__balance = balance  # Private\n    \n    @abstractmethod\n    def calculate_interest(self) -> float:\n        \"\"\"Abstract method must be implemented by subclasses\"\"\"\n        pass\n    \n    def get_balance(self) -> float:\n        \"\"\"Public method to access private balance\"\"\"\n        return self.__balance\n    \n    def _update_balance(self, amount: float) -> None:\n        \"\"\"Protected method for internal use\"\"\"\n        self.__balance += amount\n\n# 2. Inheritance - Child classes inherit from Account\nclass SavingsAccount(Account):\n    def __init__(self, account_number: str, balance: float, interest_rate: float = 0.05):\n        super().__init__(account_number, balance)  # Call parent constructor\n        self.interest_rate = interest_rate\n    \n    def calculate_interest(self) -> float:\n        \"\"\"Concrete implementation of abstract method\"\"\"\n        return self.get_balance() * self.interest_rate\n\nclass CheckingAccount(Account):\n    def __init__(self, account_number: str, balance: float, overdraft_limit: float = 500):\n        super().__init__(account_number, balance)\n        self.overdraft_limit = overdraft_limit\n    \n    def calculate_interest(self) -> float:\n        \"\"\"Different implementation for checking account\"\"\"\n        return 0.0  # No interest for checking accounts\n\n# 3. Polymorphism - Same interface, different implementations\ndef process_accounts(accounts: list[Account]) -> None:\n    \"\"\"Process different account types polymorphically\"\"\"\n    for account in accounts:\n        interest = account.calculate_interest()  # Calls appropriate method\n        balance = account.get_balance()\n        account_type = type(account).__name__\n        print(f\"{account_type}: Balance ${balance:.2f}, Interest ${interest:.2f}\")\n\n# 4. Encapsulation - Data hiding and controlled access\nclass SecureBankAccount:\n    def __init__(self, account_number: str, initial_balance: float):\n        self.__account_number = account_number  # Private\n        self.__balance = initial_balance  # Private\n        self.__transaction_history = []  # Private\n    \n    @property\n    def balance(self) -> float:\n        \"\"\"Read-only property\"\"\"\n        return self.__balance\n    \n    @property\n    def account_number(self) -> str:\n        \"\"\"Read-only property\"\"\"\n        return self.__account_number\n    \n    def deposit(self, amount: float) -> None:\n        \"\"\"Controlled access to modify balance\"\"\"\n        if amount > 0:\n            self.__balance += amount\n            self.__transaction_history.append(f\"Deposit: +${amount}\")\n        else:\n            raise ValueError(\"Deposit amount must be positive\")\n    \n    def get_transaction_history(self) -> list[str]:\n        \"\"\"Controlled access to private data\"\"\"\n        return self.__transaction_history.copy()  # Return copy to prevent modification\n\n# Demonstrate the principles\nprint(\"=== OOP Principles Demonstration ===\")\n\n# Create different account types\nsavings = SavingsAccount(\"SAV001\", 1000.0, 0.03)\nchecking = CheckingAccount(\"CHK001\", 500.0, 1000.0)\n\n# Polymorphism in action\naccounts = [savings, checking]\nprocess_accounts(accounts)\n\nprint(\"\\n=== Encapsulation Example ===\")\nsecure_account = SecureBankAccount(\"SEC001\", 1500.0)\nprint(f\"Account: {secure_account.account_number}\")\nprint(f\"Initial balance: ${secure_account.balance:.2f}\")\n\nsecure_account.deposit(200.0)\nprint(f\"After deposit: ${secure_account.balance:.2f}\")\nprint(f\"Transaction history: {secure_account.get_transaction_history()}\")\n\n# This would fail - trying to access private attribute directly\n# print(secure_account.__balance)  # AttributeError"

---\n## Object-Oriented Programming <a id=\"oop\"></a>\n\n### Question 1: What are the four main principles of OOP in Python?\n\n**Answer:** \n- **Polymorphism**: Different implementations of the same interface\n- **Inheritance**: Child classes inherit properties from parent classes\n- **Abstraction**: Hiding implementation details, exposing only necessary features\n- **Encapsulation**: Bundling data and methods, controlling access to internal state"

# Python Engineer Interview Practice Notebook

This notebook covers comprehensive Python interview questions with concise answers and clean code examples.

## Table of Contents
1. [Core Python Concepts](#core-python)
2. [Object-Oriented Programming](#oop)
3. [File Handling](#file-handling)
4. [Error Handling](#error-handling)
5. [Memory Management](#memory)
6. [Language Paradigm](#paradigm)
7. [Concurrency & Parallelism](#concurrency)
8. [API Development](#api)

---
## Core Python Concepts <a id="core-python"></a>

### Question 1: What's the difference between a list, tuple, and set in Python? When would you use each?

**Answer:** 
- **List**: Mutable, ordered, allows duplicates. Use for ordered collections that need modification.
- **Tuple**: Immutable, ordered, allows duplicates. Use for fixed data structures and as dict keys.
- **Set**: Mutable, unordered, no duplicates. Use for unique collections and fast membership testing.

In [ ]:
# List - mutable, ordered, allows duplicates
transactions = [100.0, 200.0, 150.0, 100.0]
transactions.append(300.0)
print(f"Transactions: {transactions}")

# Tuple - immutable, ordered, allows duplicates
account_info = ('ACC123', 'John Doe', 'Savings')
print(f"Account: {account_info}")

# Set - mutable, unordered, no duplicates
unique_account_types = {'Savings', 'Checking', 'Credit', 'Savings'}
print(f"Account types: {unique_account_types}")
print(f"Is 'Savings' available? {'Savings' in unique_account_types}")

### Question 2: Explain list comprehensions and provide an example

**Answer:** List comprehensions provide a concise way to create lists. They're optimized at C level, allocate memory once, and avoid temporary variables. Use generator expressions `()` for memory efficiency with large datasets.

In [ ]:
# List comprehension - memory allocated once
transactions = [100, -50, 200, -30, 150]
positive_transactions = [t for t in transactions if t > 0]
print(f"Positive transactions: {positive_transactions}")

# Generator expression - memory efficient for large datasets
large_numbers = (x * 2 for x in range(1000000) if x % 1000 == 0)
first_five = [next(large_numbers) for _ in range(5)]
print(f"First five: {first_five}")

# Banking example: Calculate fees
account_balances = [1500, 800, 2000, 300, 1200]
fees = [5 if balance < 1000 else 0 for balance in account_balances]
print(f"Account fees: {fees}")

### Question 3: What are generator expressions and when would you prefer them?

**Answer:** Generators are memory-efficient iterators that produce values on-demand using lazy evaluation. They can only be iterated once and don't support random access, but preserve memory for large datasets.

In [ ]:
def transaction_processor():
    """Generator function for processing transactions"""
    transactions = [100, 200, -50, 300, -25]
    for transaction in transactions:
        if transaction > 0:
            yield f"Credit: ${transaction}"
        else:
            yield f"Debit: ${abs(transaction)}"

# Generator expression
squared_amounts = (x**2 for x in [10, 20, 30, 40, 50])
print("Squared amounts:")
for amount in squared_amounts:
    print(amount)

# Using generator function
print("\nProcessed transactions:")
for processed in transaction_processor():
    print(processed)

# Memory comparison
import sys
list_comp = [x for x in range(1000)]
gen_exp = (x for x in range(1000))
print(f"\nList size: {sys.getsizeof(list_comp)} bytes")
print(f"Generator size: {sys.getsizeof(gen_exp)} bytes")

### Question 4: How do dictionary comprehensions work? Banking context example

**Answer:** Dictionary comprehensions create dictionaries concisely. They're atomic operations, making them thread-safe and avoiding partial states during creation.

In [ ]:
# Banking context: Account mapping
account_numbers = ['ACC001', 'ACC002', 'ACC003', 'ACC004']
balances = [1500.0, 2300.0, 890.0, 3200.0]

# Dictionary comprehension
account_balances = {acc: balance for acc, balance in zip(account_numbers, balances)}
print(f"Account balances: {account_balances}")

# Conditional dictionary comprehension
high_balance_accounts = {acc: balance for acc, balance in account_balances.items() if balance > 1000}
print(f"High balance accounts: {high_balance_accounts}")

# Transform values
account_status = {acc: 'Premium' if balance > 2000 else 'Standard' 
                 for acc, balance in account_balances.items()}
print(f"Account status: {account_status}")

### Question 5: What's the difference between `==` and `is` in Python?

**Answer:** `==` checks value equality, `is` checks object identity (same memory location). Use `==` for comparing values, `is` for comparing with `None`, `True`, `False`.

In [ ]:
# Value equality vs identity
account1 = [100, 200, 300]
account2 = [100, 200, 300]
account3 = account1

print(f"account1 == account2: {account1 == account2}")  # True - same values
print(f"account1 is account2: {account1 is account2}")  # False - different objects
print(f"account1 is account3: {account1 is account3}")  # True - same object

# Proper use of 'is'
balance = None
if balance is None:
    print("Balance not set")

# String interning example
status1 = "active"
status2 = "active"
print(f"status1 is status2: {status1 is status2}")  # True - string interning

# Object identity
print(f"id(account1): {id(account1)}")
print(f"id(account2): {id(account2)}")
print(f"id(account3): {id(account3)}")

### Question 6: Explain mutability in Python. Which types are mutable/immutable?

**Answer:** 
- **Immutable**: Cannot be changed after creation. Create new objects when modified. Examples: int, float, str, tuple, frozenset
- **Mutable**: Can be changed in-place. Examples: list, dict, set
- Immutable objects are hashable and can be dictionary keys.

In [ ]:
# Immutable types
account_number = "ACC123"
original_id = id(account_number)
account_number += "_UPDATED"  # Creates new string object
print(f"Original ID: {original_id}, New ID: {id(account_number)}")
print(f"Updated account: {account_number}")

# Mutable types
transactions = [100, 200]
original_id = id(transactions)
transactions.append(300)  # Modifies existing object
print(f"Original ID: {original_id}, After append ID: {id(transactions)}")
print(f"Transactions: {transactions}")

# Hashable custom class
class BankAccount:
    def __init__(self, account_number, balance):
        self._account_number = account_number
        self._balance = balance
    
    def __hash__(self):
        return hash((self._account_number, self._balance))
    
    def __eq__(self, other):
        if not isinstance(other, BankAccount):
            return False
        return (self._account_number == other._account_number and 
                self._balance == other._balance)
    
    def __repr__(self):
        return f"BankAccount({self._account_number}, {self._balance})"

# Using hashable objects as dictionary keys
acc1 = BankAccount("ACC001", 1000)
acc2 = BankAccount("ACC002", 2000)
account_owners = {acc1: "John Doe", acc2: "Jane Smith"}
print(f"Account owners: {account_owners}")

### Question 7: How does variable scoping work in Python? LEGB rule

**Answer:** Python follows LEGB rule for variable scope resolution:
- **L**ocal (inside function)
- **E**nclosing (inside enclosing functions) 
- **G**lobal (module level)
- **B**uilt-in (Python's built-in names)

In [ ]:
# Global scope
bank_name = "Global Bank"
interest_rate = 0.05

def calculate_compound_interest(principal, years):
    # Local scope
    local_rate = interest_rate  # Accessing global variable
    
    def inner_calculation():
        # Enclosing scope
        nonlocal local_rate
        local_rate += 0.01  # Modify enclosing scope variable
        return principal * (1 + local_rate) ** years
    
    return inner_calculation()

def banking_operations():
    global bank_name
    bank_name = "Updated Global Bank"  # Modify global variable
    
    # Local variable shadows global
    interest_rate = 0.08
    print(f"Local interest rate: {interest_rate}")
    print(f"Global interest rate: {globals()['interest_rate']}")

print(f"Before: {bank_name}")
result = calculate_compound_interest(1000, 5)
print(f"Compound interest result: {result:.2f}")

banking_operations()
print(f"After: {bank_name}")

# Built-in scope example
print(f"Built-in function: {len([1, 2, 3])}")
print(f"Built-in exception: {ValueError}")

### Question 8: What are decorators and how to create custom decorators?

**Answer:** Decorators modify or enhance functions without changing their logic directly. They wrap functions in other functions, leveraging the fact that functions are objects in Python.

In [ ]:
import functools
import time
from datetime import datetime

# Basic logging decorator
def log_calls(func):
    @functools.wraps(func)  # Preserves original function metadata
    def wrapper(*args, **kwargs):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"[{timestamp}] {func.__name__} returned: {result}")
        return result
    return wrapper

# Timing decorator
def timing_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

# Banking validation decorator
def validate_positive_amount(func):
    @functools.wraps(func)
    def wrapper(amount, *args, **kwargs):
        if amount <= 0:
            raise ValueError("Amount must be positive")
        return func(amount, *args, **kwargs)
    return wrapper

# Apply decorators
@log_calls
@timing_decorator
@validate_positive_amount
def process_transaction(amount, account_id):
    """Process a banking transaction"""
    time.sleep(0.1)  # Simulate processing time
    return f"Processed ${amount} for account {account_id}"

# Test the decorated function
try:
    result = process_transaction(100.0, "ACC123")
    print(f"Result: {result}")
    
    # This will raise an error due to validation
    process_transaction(-50.0, "ACC456")
except ValueError as e:
    print(f"Validation error: {e}")

### Question 9: Write a function to calculate average of transaction amounts

**Answer:** Use built-in functions `sum()` and `len()` with proper error handling for empty lists.

In [ ]:
def average_transactions(transactions: list[float]) -> float:
    """Calculate average of transaction amounts using built-in functions."""
    if not transactions:
        return 0.0
    
    return sum(transactions) / len(transactions)

# Alternative with statistics module
import statistics

def average_transactions_stats(transactions: list[float]) -> float:
    """Calculate average using statistics module."""
    return statistics.mean(transactions) if transactions else 0.0

# Test both approaches
test_transactions = [100.50, 250.75, -50.25, 300.00, 75.20]
empty_transactions = []

print(f"Transactions: {test_transactions}")
print(f"Average (built-in): {average_transactions(test_transactions):.2f}")
print(f"Average (statistics): {average_transactions_stats(test_transactions):.2f}")

print(f"\nEmpty list average: {average_transactions(empty_transactions)}")

# Additional banking calculations
def transaction_summary(transactions: list[float]) -> dict:
    """Generate comprehensive transaction summary."""
    if not transactions:
        return {"count": 0, "total": 0.0, "average": 0.0, "max": 0.0, "min": 0.0}
    
    return {
        "count": len(transactions),
        "total": sum(transactions),
        "average": sum(transactions) / len(transactions),
        "max": max(transactions),
        "min": min(transactions)
    }

summary = transaction_summary(test_transactions)
print(f"\nTransaction summary: {summary}")

### Question 10: How to use `enumerate()` to process transactions with indices?

**Answer:** `enumerate()` returns tuples of (index, item) for each element, starting from 0 by default. Useful for tracking position while iterating.

In [ ]:
def process_transactions_with_index(transactions: list[float]) -> list[tuple]:
    """Process transactions with their indices using enumerate."""
    processed = []
    
    for index, transaction in enumerate(transactions):
        status = "Credit" if transaction > 0 else "Debit"
        processed.append((index, transaction, status))
    
    return processed

# Alternative with list comprehension
def process_transactions_comprehension(transactions: list[float]) -> list[tuple]:
    """Process using list comprehension with enumerate."""
    return [(i, amount, "Credit" if amount > 0 else "Debit") 
            for i, amount in enumerate(transactions)]

# Banking example: Flag suspicious transactions
def flag_suspicious_transactions(transactions: list[float], threshold: float = 1000.0) -> dict:
    """Flag transactions above threshold with their positions."""
    suspicious = {}
    
    for index, amount in enumerate(transactions, start=1):  # Start from 1
        if abs(amount) > threshold:
            suspicious[f"Transaction_{index}"] = {
                "amount": amount,
                "position": index,
                "type": "Large Credit" if amount > 0 else "Large Debit"
            }
    
    return suspicious

# Test the functions
sample_transactions = [250.50, -100.25, 1500.00, 75.00, -2000.00, 300.75]

print("Original transactions:", sample_transactions)
print("\nProcessed with indices:")
for item in process_transactions_with_index(sample_transactions):
    print(f"  Index {item[0]}: ${item[1]} - {item[2]}")

print("\nProcessed with comprehension:")
processed_comp = process_transactions_comprehension(sample_transactions)
for item in processed_comp:
    print(f"  Index {item[0]}: ${item[1]} - {item[2]}")

print("\nSuspicious transactions:")
suspicious = flag_suspicious_transactions(sample_transactions)
for trans_id, details in suspicious.items():
    print(f"  {trans_id}: ${details['amount']} at position {details['position']} - {details['type']}")